# Imports and consts

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from eda_utils.plotting import plot_cat_counts

ModuleNotFoundError: No module named 'utils'

In [10]:
DATA_PATH = 'Data/'
FILE_NAME = 'riskified_ds_home_task_dataset_(3).csv'

# Load and Explore Data

## Load

In [15]:
raw_df = pd.read_csv(f'{DATA_PATH}{FILE_NAME}', parse_dates=['created_at'])
raw_df.shape

C:\Users\uvors\AppData\Local\Temp\ipykernel_6536\4229271681.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(f'{DATA_PATH}{FILE_NAME}', parse_dates=['created_at'])


(190000, 21)

In [12]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190000 entries, 0 to 189999
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   order_id                 190000 non-null  int64  
 1   status                   190000 non-null  object 
 2   email_anoni              190000 non-null  object 
 3   billing_country_code     189223 non-null  object 
 4   shipping_country_code    189721 non-null  object 
 5   shipping_method          185461 non-null  object 
 6   created_at               190000 non-null  object 
 7   total_spent              190000 non-null  float64
 8   currency_code            190000 non-null  object 
 9   gateway                  189862 non-null  object 
 10  V1_link                  189993 non-null  object 
 11  V2_distance              157964 non-null  float64
 12  V3_distance              158957 non-null  float64
 13  V4_our_age               189993 non-null  float64
 14  V5_m

## Destributions

### Numerical

In [13]:
raw_df.describe()

,order_id,total_spent,V2_distance,V3_distance,V4_our_age,V5_merchant_age,V8_ip,V9_cookie,V10_cookie,V11_cookie
count,1.900000e+05,190000.000000,157964.000000,158957.000000,189993.000000,189993.000000,185132.000000,189816.000000,188148.000000,111295.000000
mean,1.589366e+08,262.806693,1176.859443,1262.328208,653.947451,216.220645,0.076579,82.207796,5.305185,47.386270
std,3.234140e+07,497.157831,3088.102927,3206.620899,2046.051781,6976.480661,0.387298,589.419776,9.227724,114.170882
min,2.179327e+07,0.000000,0.000000,0.000000,0.000000,-323.000000,0.000000,0.000000,0.000000,-140.884769
25%,1.319893e+08,77.400000,4.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.631284,0.016134
50%,1.576925e+08,140.400000,13.000000,13.000000,51.000000,1.000000,0.000000,4.000000,1.533317,1.773900
75%,1.871457e+08,267.000000,174.000000,198.000000,692.000000,82.000000,0.000000,18.000000,4.443918,36.878183
max,2.160862e+08,57746.005396,19610.000000,19610.000000,730603.000000,736666.000000,4.000000,24260.000000,100.000000,1181.605035


Ovservations:
1. Total spend has 0 - Declined?
2. Our age - at least 25% are 0. NaN equivilant?
3. Mechant age has negative values. 50% are <=1
4. V8_ip looks categorical
5. Suspected outliers of extreme values - all but v8_ip

In [ ]:
ORDINAL_CAT_COLS = ['V8_ip']

### str/cat

In [16]:
raw_df.select_dtypes(include=['object','category']).describe()

,status,email_anoni,billing_country_code,shipping_country_code,shipping_method,currency_code,gateway,V1_link,V6_avs_result,V7_bill_ship_name_match
count,190000,190000,189223,189721,185461,190000,189862,189993,189993,189993
unique,3,189946,213,195,2367,35,255,2,7,4
top,approved,NA@personal_domain.com,US,US,value_1645,USD,value_65,False,Unsupported,FL
freq,165441,30,65900,71482,20896,124681,28344,188851,68569,167300


Observations:
1. Most common email is the NA - i.e. this is not a specific 
2. billing_country_code, shipping_country_code both have ~200 uniques so maybe aggregate somehow. Also gateway.
2. Shipping method has 2367 nunique so consider how to handle - maybe one-hot is not so good.
3. V1_link looks boolean so check why it's object - probably NaNs.
4. Look at 'V6_avs_result' and see if unsupported is a special value. Either way it has 7 uniques.

In [17]:
STR_CAT_COLS = ['status', 'billing_country_code', 'shipping_country_code', 'shipping_method', 'currency_code', 'V6_avs_result', 'V7_bill_ship_name_match'] # To be one-hot encoded
STR_ORDINAL_COLS = [] # To be label encoded